# Imports

In [1]:
import os
import numpy as np
import pandas as pd
import shutil
from gzip import open as gopen
import subprocess
from concurrent.futures import ProcessPoolExecutor
# seed the rng
np.random.seed(42)

In [2]:
# go through each simulation directory to generate corrected transmission networks, sample data and time trees
for i in range(1,1101):
    # generate latent period
    latent_period = np.random.exponential(2.9/365) #expected value is 2.9/365 days
    # set up file paths for transmission network
    old_tn_path = f'simulations/{i:04d}/transmission_network.subsampled.txt.gz'
    new_tn_path = f'simulations/{i:04d}/transmission_network.subsampled.corrected.txt'
    # open files and write out the corrected data
    with gopen(old_tn_path, 'rt') as old_tn, open(new_tn_path, 'w') as new_tn:
        for line in old_tn:
            u, v, t = line.strip().split('\t')
            new_tn.write(f'{u}\t{v}\t{float(t) + latent_period:.6f}\n')
    # set up file paths for sample times
    old_samples_path = f'simulations/{i:04d}/subsample_times.txt.gz'
    new_samples_path = f'simulations/{i:04d}/subsample_times.corrected.txt'
    # open files and write out the corrected data
    with gopen(old_samples_path, 'rt') as old_samples, open(new_samples_path, 'w') as new_samples:
        for line in old_samples:
            u, t = line.strip().split('\t')
            new_samples.write(f'{u}\t{float(t) + latent_period}\n')
    # setup and run CoaTRan to generate corrected time trees
    command = ['coatran_constant', new_tn_path, new_samples_path, '1']
    env = os.environ.copy()
    coatran_rng_seed = np.random.randint(low=0, high=2**31) # ensure it is reproducible
    env["COATRAN_RNG_SEED"] = str(coatran_rng_seed)
    result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, env=env)
    if result.returncode != 0:
        print(f"Error executing CoaTran for simulation {i}: {result.stderr.decode()}")
        print(command)
    else:
        tree_path = f'simulations/{i:04d}/tree.time.subsampled.corrected.nwk'
        with open(tree_path, 'w') as file_handle:
            file_handle.write(result.stdout.decode())

In [3]:
# Setup output folder for each of 1000 resamples
# each with subfolders for CC and AB clade analysisiresults
if os.path.exists('resamples'):
    # Remove the directory and its contents
    shutil.rmtree('resamples')
os.makedirs('resamples')
for i in range(1000):
    # create path for this resample
    os.makedirs(f'resamples/{i}')
    # create subdirectory with clade analysis results subdirectories
    os.makedirs(f'resamples/{i}/clade_analyses_CC')
    os.makedirs(f'resamples/{i}/clade_analyses_AB')

In [4]:
# function to call stableCoalescence_cladeAnalysis.py
# to resample one simulation 1000 times
def resample_simulation(i, seed):
    command = [
        "python3",
        "stableCoalescence_cladeAnalysis.py",
        "-tn", f'simulations/{i:04d}/transmission_network.subsampled.corrected.txt',
        "-tt", f'simulations/{i:04d}/tree.time.subsampled.corrected.nwk',
        "-g", f'simulations/{i:04d}/GEMF_files/output.txt.gz',
        "-s", "0.00092",
        "-m", "1",
        "-id", f'{i:04d}',
        "-seed", str(seed)
    ]
    subprocess.run(command)

In [5]:
# resample each of the 1100 simulations with a deterministic seed
seeds = np.random.randint(0, 2**31, size=1100)
with ProcessPoolExecutor(max_workers=24) as executor:
    executor.map(resample_simulation, range(1, 1101), seeds)

# Primary analysis (3.47-day doubling time, 15% ascertainment rate) tree sizes

In [6]:
def read_clade_results(dir):
    clade_analyses_d = dict()
    for path in sorted(os.listdir(dir)): # pool all clade analyses together
        clade_analysis_path = dir + path
        key = int(path.split('_')[0])
        clade_analyses_d[key] = {'clade_sizes': [], 'subclade_sizes': []}
        for line in open(clade_analysis_path):
            l = line.strip().strip(']').split('[')
            clade_size = int(l[0].strip()) # make each clade size (including single leaves) an integer
            subclade_sizes = [int(x) for x in l[1].strip().replace(' ', '').split(',')] # put each subclade size (including single leaves) into a list
            clade_analyses_d[key]['clade_sizes'].append(clade_size)
            clade_analyses_d[key]['subclade_sizes'].append(subclade_sizes)
    return clade_analyses_d

In [7]:
clade_analyses_CC_dir = './resamples/0/clade_analyses_CC/'
clade_analyses_AB_dir = './resamples/0/clade_analyses_AB/'

clade_analyses_CC_d = read_clade_results(clade_analyses_CC_dir)

print('Less than 787 taxa: %f' % (sum([sum(clade_analyses_CC_d[x]['clade_sizes']) < 787 for x in clade_analyses_CC_d])/1100)) 
print('More than 1000 taxa: %f' % (sum([sum(clade_analyses_CC_d[x]['clade_sizes']) > 1000 for x in clade_analyses_CC_d])/1100))
print('More than 5000 taxa: %f' % (sum([sum(clade_analyses_CC_d[x]['clade_sizes']) > 5000 for x in clade_analyses_CC_d])/1100))

Less than 787 taxa: 0.017273
More than 1000 taxa: 0.980909
More than 5000 taxa: 0.953636


# Tree shapes and bayes factors

In [8]:
unconstrained_results = np.array([1.68,80.85, 10.32, 0.92])/100 # linB, linA, C/C, T/T
recCA_results = np.array([77.28, 8.18, 10.49, 3.71])/100 # linB, linA, C/C, T/T

In [9]:
def calculate_bf(asr_results, simulation_results):
    # Let t_p be a polytomy, t_1C be one clade, and t_2c be two clades. Note that t_p is a topology with a polytomy but is not t_1c. t_p + t_1c equals all topologies with a basal polytomy (Fig. 2a). 
    # trees are in the order
    # (t_p, t_1C, t_2C, (t_p,(t_p,t_1C,t_2C)), (t_1C,(t_p,t_1C,t_2C)), (t_2c,(t_p,t_1C,t_2C)))
    compatibility_matrix = np.array([np.array([0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0]), # S_A
                                     np.array([0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0]), # S_B
                                     np.array([0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0]), # S_CC
                                     np.array([0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0])] # S_TT
                                    )
                                    
    # A matrix of conditional probabilities
    # Each row stores the vector Pr(S_MRCA | \btau)
    pr_s_mrca_given_tree = np.array([x/sum(x) if sum(x) > 0 else x for x in compatibility_matrix.T]) # if tree not associated with any haplotype, just keep the row as all 0s
    
    # Order: S_A, S_B, S_{C/C}, S_{T/T}
    pr_s_mrca_given_data = np.array(asr_results)/sum(asr_results)
    unnormalized_pr_data_given_s_mrca = pr_s_mrca_given_data.copy()
    
    # FAVITES simulation information
    # the 3 trees are in the order (t_p, t_1C, t_2C)
    pr_3_topos = np.array(simulation_results)
    pr_trees_given_I1 = np.concatenate([pr_3_topos, np.array([0]*9)])
    pr_trees_given_I2 = np.concatenate([np.array([0]*3), np.outer(pr_3_topos, pr_3_topos).flatten()])
    
    # Equal prior probability of 1 or 2 intros
    pr_I1 = 0.5
    pr_I2 = 0.5
    
    pr_s_mrca_and_I1 = np.dot([np.dot(pr_s_mrca_given_tree.T[i], pr_trees_given_I1) for i in range(0,4)], pr_I1) # dot product of P(haplotype|tree) (column) and P(trees|I_n), scaled by P(I_n)
    pr_s_mrca_and_I2 = np.dot([np.dot(pr_s_mrca_given_tree.T[i], pr_trees_given_I2) for i in range(0,4)], pr_I2)

    posterior_odds = np.dot(unnormalized_pr_data_given_s_mrca, pr_s_mrca_and_I2) / np.dot(unnormalized_pr_data_given_s_mrca, pr_s_mrca_and_I1)
    prior_odds = pr_I2/pr_I1
    BF = posterior_odds/prior_odds

    return(BF)



In [10]:
def clade_analysis_updated(clade_analyses_CC_dir, clade_analyses_AB_dir, label, min_polytomy_size=100, _print_=False):
    # C/C analyses focus on 1-mutation clades
    # therefore, all descendant lineages are included:
    #   -1 = unmutated leaf
    #    1 = mutated leaf separate from a clade
    #   >1 = clade
    clade_analyses_CC_d = read_clade_results(clade_analyses_CC_dir)

    # A/B analyses focus on 2-mutation clades
    # only 2-mutation clades are included 
    clade_analyses_AB_d = read_clade_results(clade_analyses_AB_dir)
    
    # C/C analysis
    cc_count_30perc_twoPolytomies = 0 # how often are there only two 1-mutation clades, each constituting more than 30% of taxa AND with a polytomy at the base of each clade, and no other descendants from the root?
    for run in clade_analyses_CC_d:
        clade_sizes = clade_analyses_CC_d[run]['clade_sizes'] # get the clade sizes
        subclade_sizes = [len(x)>=min_polytomy_size for x in clade_analyses_CC_d[run]['subclade_sizes']] # check if each clade has a polytomy at the base
        if len(clade_sizes) == 2: # if there are more than two descendants, including individual leaves, skip
            if min(clade_sizes) > (sum(clade_sizes)*0.30): # make sure each clade is >30%
                if False not in subclade_sizes: # each clade must have a polytomy at the base
                    cc_count_30perc_twoPolytomies += 1
    
    # A/B analysis
    ab_count_30perc_twoPolytomies = 0 # interested in 2 mutations clade that are at least 30% of all taxa + has a basal polytomy + polytomy at 2 mutation clade
    lower_constraint = 0.3 # the 2-mutation clade must be at least 30% of all taxa
    upper_constraint = 0.7 # the 2-mutation clade must be at most 70% of all taxa

    for run in clade_analyses_AB_d:
        num_leaves = sum(clade_analyses_CC_d[run]['clade_sizes'])
        base_polytomy_size = len(clade_analyses_CC_d[run]['clade_sizes']) # check how many lineages descend from the root
        clade_sizes = clade_analyses_AB_d[run]['clade_sizes'] 
        subclade_sizes = clade_analyses_AB_d[run]['subclade_sizes']
        if not clade_sizes: # no 2 mutation clades
            continue
        for index, clade_size in enumerate(clade_sizes): # loop through all 2 mutation clades
            if lower_constraint*num_leaves <= clade_size <= upper_constraint*num_leaves: # clade match size restrictions
                if base_polytomy_size >= min_polytomy_size: # basal polytomy
                    if len(subclade_sizes[index]) >= min_polytomy_size: # polytomy at 2 mutation clade
                        ab_count_30perc_twoPolytomies += 1 # if all conditions are met, add 1 to the count
                        break # if one 2 mutation clade meets the conditions, break out of the loop and move on to the next run

    # polytomy: checking if there is a polytomy at the base of the tree; can do this using the C/C analysis
    min_polytomy_descendants = min_polytomy_size
    count_atLeastMinDescendants = 0
    for run in clade_analyses_CC_d: # loop through all runs
        clade_sizes = clade_analyses_CC_d[run]['clade_sizes'] # get the clade sizes
        if len(clade_sizes) >= min_polytomy_descendants:  # if there are at least X number of descendants, including individual leaves
            count_atLeastMinDescendants += 1
    polytomy_result = count_atLeastMinDescendants/1100
    
    # calculate bayes factors
    cc_result = cc_count_30perc_twoPolytomies/1100
    ab_result = ab_count_30perc_twoPolytomies/1100

    simulation_results = [polytomy_result-ab_result, ab_result, cc_result] # note that polytomy result is inclusive of ab_result, so subtract ab_result from polytomy_result to avoid double counting
    unconstrained_results = np.array([1.68, 80.85, 10.32, 0.92])/100 # linA, linB, C/C, T/T
    recCA_results = np.array([77.28, 8.18, 10.49, 3.71])/100 # linA, linB, C/C, T/T
    bf_unconstrained = calculate_bf(unconstrained_results, simulation_results)
    bf_recCA = calculate_bf(recCA_results, simulation_results)
    
    if _print_ == True:
        print('Proportion with a polytomy at the base: %f\n' % (count_atLeastMinDescendants/1100))
        print('C/C clade analysis: 2 clades, polytomy at base of each clade, each clade possessing at least 30%% of the taxa: %f\n' % (cc_count_30perc_twoPolytomies/1100))
        print('A/B clade analysis: Basal polytomy, polytomy at 2-mut clade, with the clade possessing between 30 and 70%% of the taxa: %f\n' % (ab_count_30perc_twoPolytomies/1100))
        print('Unconstrained Bayes factor: %f' % bf_unconstrained)
        print('recCA Bayes factor: %f' % bf_recCA)

    return [cc_count_30perc_twoPolytomies, ab_count_30perc_twoPolytomies, count_atLeastMinDescendants, bf_unconstrained, bf_recCA]


### Main result

In [11]:
# collect clade analysis results
def process_clade_analysis(i):
    clade_analyses_CC_dir = f'./resamples/{i}/clade_analyses_CC/'
    clade_analyses_AB_dir = f'./resamples/{i}/clade_analyses_AB/'
    return clade_analysis_updated(clade_analyses_CC_dir, clade_analyses_AB_dir, '3.5 DT', min_polytomy_size=100)

# Initialize lists to store results
cc_counts = []
ab_counts = []
polytomy_counts = []
bf_unconstrained_results = []
bf_recCA_results = []

with ProcessPoolExecutor(max_workers=24) as executor:
    # Map process_clade_analysis function across the range of values
    results = executor.map(process_clade_analysis, range(1000))

    # Unpack the results and append them to the respective lists
    for a, b, c, d, e in results:
        cc_counts.append(a)
        ab_counts.append(b)
        polytomy_counts.append(c)
        bf_unconstrained_results.append(d)
        bf_recCA_results.append(e)

In [12]:
# write out means
# Calculate mean counts
mean_cc_count = np.mean(cc_counts)
mean_ab_count = np.mean(ab_counts)
mean_polytomy_count = np.mean(polytomy_counts)

# Convert means to topology likelihoods for the Bayes factor calculation
mean_simulation_results = np.array([mean_polytomy_count - mean_ab_count, mean_ab_count, mean_cc_count]) / 1100

# Setup the MRCA haplotype posterior probabilities for the Bayes factor calculation
unconstrained_results = np.array([1.68, 80.85, 10.32, 0.92])/100  
recCA_results = np.array([77.28, 8.18, 10.49, 3.71])/100  

# Calculate Bayes factors of expected parameters
bf_unconstrained = calculate_bf(unconstrained_results, mean_simulation_results)
bf_recCA = calculate_bf(recCA_results, mean_simulation_results)

# Calculate expected Bayes factors
mean_unc = np.mean(bf_unconstrained_results)
mean_recCA = np.mean(bf_recCA_results)

with open('mean_resample_results_corrected.txt', 'w') as f:
    f.write(f'expected CC rate: {mean_cc_count*100/1100:.2f}%\n')
    f.write(f'expected AB rate: {mean_ab_count*100/1100:.2f}%\n')
    f.write(f'expected polytomy rate: {mean_polytomy_count*100/1100:.2f}%\n')
    f.write(f'expected BF (unconstrained): {mean_unc:.1f}\n')
    f.write(f'expected BF (recCA): {mean_recCA:.1f}\n')
    f.write(f'BF of expected rates (unconstrained): {bf_unconstrained:.1f}\n')
    f.write(f'BF of exptected rates (recCA): {bf_recCA:.1f}\n')
    
data = [
    ['3.47','0.15',
     f'{mean_polytomy_count*100/1100:.2f}',
     f'{mean_cc_count*100/1100:.2f}', 
     f'{mean_ab_count*100/1100:.2f}', 
     f'{mean_unc:.1f}', 
     f'{mean_recCA:.1f}']
]
columns = pd.MultiIndex.from_tuples([
    ('Analysis', 'Doubling Time (days)'),
    ('Analysis', 'Ascertainment Rate'),
    ('Frequency', 'Basal Polytomy (%)'),
    ('Frequency', 'CC Topology (%)'),
    ('Frequency', 'AB Topology (%)'),
    ('Bayes Factor', 'unconstrained'),
    ('Bayes Factor', 'recCA'),
])
df_multi_level = pd.DataFrame(data, columns=columns)
df_multi_level

Analysis                             Frequency                  \
  Doubling Time (days) Ascertainment Rate Basal Polytomy (%) CC Topology (%)   
0                 3.47               0.15              44.32            0.17   

                   Bayes Factor        
  AB Topology (%) unconstrained recCA  
0            3.34           3.4   3.5